<a href="https://colab.research.google.com/github/JvSecate/PDI/blob/main/Atividades/3%20-%20Transformações%20de%20intensidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir images
!wget -q -N -P images https://raw.githubusercontent.com/JvSecate/PDI//main/Atividades/images/Fig0308_fractured_spine.tif
!wget -q -N -P images https://raw.githubusercontent.com/JvSecate/PDI//main/Atividades/images/enhance-me.gif

##Desenvolver código para esteganografia em imagem tom de cinza e colorida.

##Obter imagens (Colab)

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from PIL import Image, ImageOps
import math

###Converter Mensagem para binário

In [2]:
def text_to_bits(text):
  bits = ''.join(format(ord(c), '08b') for c in text)
  return bits
def bits_to_text(bits):
  chars = [bits[i:i+8] for i in range(0, len(bits), 8)]
  return ''.join([chr(int(char, 2)) for char in chars])

###Encodar mensagem

In [3]:
def Encode(imagem, texto, saida):
  img = Image.open(imagem)
  img_array = np.array(img)
  text_bits = text_to_bits(texto) + '1111111111111110'
  text_bits = list(text_bits)
  num_pixels = img_array.size

  if len(text_bits) > num_pixels:
    raise ValueError("O texto é muito grande para ser armazenado nesta imagem.")

  flat_array = img_array.flatten()
  for i in range(len(text_bits)):
    flat_array[i] = (flat_array[i] & 0xFE) | int(text_bits[i])

  img_array = flat_array.reshape(img_array.shape)
  resultado = Image.fromarray(img_array)
  resultado.save(saida)

  return resultado

def Decode(imagem):
  img = Image.open(imagem)
  img_array = np.array(img)
  flat_array = img_array.flatten()
  bits = ''
  for i in range(len(flat_array)):
    bits += str(flat_array[i] & 1)
    if bits[-16:] == '1111111111111110':
      break
  bits = bits[:-16]
  return bits_to_text(bits)


##Exercícios - Transformações e Histograma


#####Utilizar as imagens [Fig 3.8](https://ead.bri.ifsp.edu.br/moodle/mod/resource/view.php?id=72359) e [enhance-me](https://ead.bri.ifsp.edu.br/moodle/mod/resource/view.php?id=72360) disponíveis no Moodle.




###Aplicar a transformação logarítmica, testar vários valores para o parâmetro c "s = c log (1 + r)"

In [4]:
def Logaritmo(imagem, c=1):
  image = Image.open(imagem)
  imagem_np = np.array(image)
  log = c * np.log(1 + imagem_np)
  log = np.uint8(255 *log / np.max(log))
  return log

###Aplicar a transformação de potência (gama), testar vários valores para o parâmetro γ e c=1 "s = crγ"

In [5]:
def Potencia(imagem, gamma=1, c=1):
  image = Image.open(imagem)
  imagem_np = np.array(image)
  resultado = c * (np.float32(imagem_np) ** gamma)
  resultado = np.uint8(resultado * 255 / np.max(resultado))
  return resultado

###Implemente a representação de cada plano de bits das imagens.


In [6]:
def PlanoDeBits(imagem):
  image = Image.open(imagem)
  imagem_np = np.array(image)
  resultado = []
  for i in range(8):
    bit_plane = (imagem_np >> i) & 1
    resultado.append(bit_plane * 255)
  fig, axes = plt.subplots(2, 4, figsize=(12, 6.3))

  fig.suptitle('Planos de Bits da Imagem cinza e colorida')
  for i, ax in enumerate(axes.flat):
    ax.imshow(resultado[i], cmap='gray')
    ax.set_title(f'Plano de Bit {i}')
    ax.axis('off')
  plt.show()

  for i in range(8):
    bit_plane = (imagem_np >> i) & 1
    resultado.append(bit_plane * 255)
  fig, axes = plt.subplots(2, 4, figsize=(12, 6.3))

  for i, ax in enumerate(axes.flat):
    ax.imshow(resultado[i])
    ax.set_title(f'Plano de Bit {i}')
    ax.axis('off')
  plt.show()

###Implementar a equalização do histograma manualmente e utilizando bibliotecas: OpenCV, Pillow, numpy, etc;

In [9]:
def Equalizacao_manual(imagem):
  image = Image.open(imagem)
  imagem_np = np.array(image)
  hist, bins = np.histogram(imagem_np.flatten(), 256, [0,256])
  cdf = hist.cumsum()
  cdf_normalized = cdf * hist.max() / cdf.max()
  cdf_m = np.ma.masked_equal(cdf, 0)
  cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
  resultado = cdf[imagem_np]
  return resultado

def Equalizacao_opencv(imagem):
  image = Image.open(imagem)
  imagem_np = np.array(image)
  resultado = cv2.equalizeHist(imagem_np)
  return resultado

def Equalizacao_pillow(imagem):
  image = Image.open(imagem)
  resultado = ImageOps.equalize(image)
  return resultado

def Equalizacao_numpy(imagem):
  image = Image.open(imagem)
  imagem_np = np.array(image)
  hist, bins = np.histogram(imagem_np.flatten(), 256, [0, 256])
  cdf = hist.cumsum()
  cdf_normalized = cdf * hist.max() / cdf.max()
  cdf_m = np.ma.masked_equal(cdf, 0)
  cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
  resultado = cdf[imagem_np]
  return resultado